In [2]:
import pandas as pd

malear = pd.read_csv('Male/males_ar.csv')
maleen = pd.read_csv('Male/males_en.csv')
mennames = pd.read_csv('Male/Men-Names.txt')

mennames['Gender'] = "Male"

males = pd.concat([malear, maleen, mennames], axis=0)

femalear = pd.read_csv('Female/females_ar.csv')
femaleen = pd.read_csv('Female/females_en.csv')
womannames = pd.read_csv('Female/Women-Names.txt')

womannames['Gender'] = "Female"

females = pd.concat([femalear, femaleen, womannames])

arabnames = pd.read_csv('Both/arabic_names.csv')

arab = pd.concat([arabnames['arabic_name'], arabnames['gender']], axis=1)
english = pd.concat([arabnames['english_name'], arabnames['gender']], axis=1)

arab.rename(columns={'gender': 'Gender', 'arabic_name': 'Name'}, inplace=True)
english.rename(columns={'gender': 'Gender', 'english_name': 'Name'}, inplace=True)

secondarabnames = pd.read_csv('Both/arabnames.csv')

thirdarabnames = pd.read_csv('Both/KDBMUSLIM.txt', sep='\t')
thirdnamesenglish = pd.concat([thirdarabnames['Roman'], thirdarabnames['Gender']], axis=1)
thirdnamesarabic = pd.concat([thirdarabnames['Reference'], thirdarabnames['Gender']], axis=1)

thirdnamesenglish.rename(columns={'Roman': 'Name'}, inplace=True)
thirdnamesarabic.rename(columns={'Reference': 'Name'}, inplace=True)

df = pd.concat([males,
                females,
                arab, 
                english, 
                secondarabnames, 
                thirdnamesenglish, 
                thirdnamesarabic], axis=0)

df = df.drop_duplicates().dropna()

df['Gender'] = df['Gender'].replace(r'\bm\b', 'Male', regex=True)
df['Gender'] = df['Gender'].replace(r'\bf\b', 'Female', regex=True)
df['Gender'] = df['Gender'].replace(r'\bF\b', 'Female', regex=True)
df['Gender'] = df['Gender'].replace(r'\bM\b', 'Male', regex=True)

df = df[(df['Gender'] == 'Male') | (df['Gender'] == 'Female')]

df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

In [20]:
# 👇 لازم يكون أول شيء
import torch, numpy

# السماح بالـ pickle الداخلي القديم
torch.serialization.add_safe_globals([numpy.core.multiarray._reconstruct])

# ✅ نجعل التحميل في Stanza يعمل دائمًا بـ weights_only=False
_old_load = torch.load
def _patched_load(*args, **kwargs):
    kwargs["weights_only"] = False
    return _old_load(*args, **kwargs)
torch.load = _patched_load

C:\Users\abeda\AppData\Local\Temp\ipykernel_6920\2946150685.py:5: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  torch.serialization.add_safe_globals([numpy.core.multiarray._reconstruct])


In [21]:
import spacy_stanza

en = spacy_stanza.load_pipeline('en')
ar = spacy_stanza.load_pipeline('ar')

2025-11-20 15:21:11 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-11-20 15:21:15 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2025-11-20 15:21:15 WARNING: GPU requested, but is not available!
2025-11-20 15:21:15 INFO: Using device: cpu
2025-11-20 15:21:15 INFO: Loading: tokenize
2025-11-20 15:21:15 INFO: Loading: pos
2025-11-20 15:21:16 INFO: Loading: lemma
2025-11-20 15:21:16 INFO: Loading: constituency
2025-11-20 15:21:17 INFO: Loading: depparse
2025-11-20 15:21:17 INFO: Loading: sentiment
2025-11-20 15:21:18 INFO: Loading: ner
2025-11-20 15:21:19 INFO: Done loading processors!
2025-11-20 15:21:19 INFO: Checking for updates to resources.json in case models have been updated.  Note: this b

2025-11-20 15:21:24 INFO: Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

2025-11-20 15:21:24 WARNING: GPU requested, but is not available!
2025-11-20 15:21:24 INFO: Using device: cpu
2025-11-20 15:21:24 INFO: Loading: tokenize
2025-11-20 15:21:25 INFO: Loading: mwt
2025-11-20 15:21:25 INFO: Loading: pos
2025-11-20 15:21:25 INFO: Loading: lemma
2025-11-20 15:21:25 INFO: Loading: depparse
2025-11-20 15:21:26 INFO: Loading: ner
2025-11-20 15:21:28 INFO: Done loading processors!


In [17]:
from langdetect import detect
import numpy as np

def preprocess(text):
    nlp = None

    if detect(text) == 'ar':
        nlp = ar(text)
    else:
        nlp = en(text)    

    return [token.vector for token in nlp]   

In [6]:
from googletrans import Translator
import pandas as pd
import time

async def translateanaddmore(series, container):
    translator = Translator()
    
    for i, word in enumerate(series):
        try:
            # إضافة تأخير لتجنب حظر IP
            if i > 0:
                time.sleep(0.5)
            
            # ✅ التصحيح: الكشف عن اللغة بشكل صحيح
            detected = (await translator.detect(word))
            
            # السمات الصحيحة للكائن detected
            detected_lang = detected.lang  # هذه يجب أن تعمل
            
            # الترجمة بناءً على اللغة المكتشفة
            if detected_lang == 'ar':
                translation = await translator.translate(word, src='ar', dest='en')
            else:
                translation = await translator.translate(word, src='en', dest='ar')
            
            translated_text = translation.text
            
            # باقي الكود...
            if translated_text not in container['Name'].values:
                if word in container['Name'].values:
                    Gender = container[container['Name'] == word]['Gender'].values[0]
                    
                    new_row = pd.DataFrame({'Name': [translated_text], 'Gender': [Gender]})
                    container = pd.concat([container, new_row], ignore_index=True)
                    print(f"✅ تمت إضافة: '{translated_text}' من '{word}'")
                else:
                    print(f"⚠️ الكلمة الأصلية '{word}' غير موجودة في container")
            else:
                print(f"ℹ️ الترجمة '{translated_text}' موجودة مسبقاً")
                
        except Exception as e:
            print(f"❌ خطأ في معالجة '{word}': {e}")
            continue
    
    return container

In [9]:
df.shape

(16323, 2)

In [10]:
df = await translateanaddmore(df['Name'], df)

ℹ️ الترجمة 'آدم' موجودة مسبقاً
✅ تمت إضافة: 'captivating' من 'آسر'
✅ تمت إضافة: 'Never' من 'أبد'
✅ تمت إضافة: 'Get creative' من 'أبدع'
✅ تمت إضافة: 'needle' من 'أبره'
✅ تمت إضافة: 'The most valiant' من 'أبسل'
✅ تمت إضافة: 'I smile' من 'أبسم'
✅ تمت إضافة: 'Preach' من 'أبشر'
✅ تمت إضافة: 'I am brilliant' من 'أبلج'
✅ تمت إضافة: 'my dad' من 'أبي'
✅ تمت إضافة: 'shred' من 'أجاد'
✅ تمت إضافة: 'Thank you' من 'أجزل'
✅ تمت إضافة: 'More beautiful' من 'أجمل'
✅ تمت إضافة: 'I answer' من 'أجوب'
✅ تمت إضافة: 'The finest' من 'أجود'
✅ تمت إضافة: 'Calculate' من 'أحسب'
✅ تمت إضافة: 'better' من 'أحسن'
✅ تمت إضافة: 'Be modest' من 'أحشم'
ℹ️ الترجمة 'Ahmed' موجودة مسبقاً
✅ تمت إضافة: 'I deviate' من 'أحيد'
✅ تمت إضافة: 'Akhzam' من 'أخزم'
✅ تمت إضافة: 'last' من 'أخير'
✅ تمت إضافة: 'Achilles' من 'أخيل'
✅ تمت إضافة: 'A.D.D' من 'أُدد'
✅ تمت إضافة: 'I reply' من 'أدرد'
✅ تمت إضافة: 'Adram' من 'أدرم'
✅ تمت إضافة: 'Assimilate' من 'أدغم'
✅ تمت إضافة: 'Adlam' من 'أدلم'
✅ تمت إضافة: 'Adham' من 'أدهم'
✅ تمت إضافة: 'writer

In [11]:
df.shape

(25083, 2)

In [12]:
df.to_csv('AllDataAfterTranslate.csv')

In [13]:
df['Name'] = df['Name'].apply(preprocess)

df.to_csv('AllDataAfterConvertingToVectorsAfterTranslate.csv')

c:\Users\abeda\AppData\Local\Programs\Python\Python313\Lib\site-packages\spacy\language.py:1041: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  doc = self._ensure_doc(text)
c:\Users\abeda\AppData\Local\Programs\Python\Python313\Lib\site-packages\spacy\language.py:1041: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['و', 'أبل']
Entities: []
  doc = self._ensure_doc(text)
c:\Users\abeda\AppData\Local\Programs\Python\Python313\Lib\site-packages\spacy\language.py:1041: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['و', 'أدع']
Entities: []
  doc = self._ensure_doc(text)
c:\Users\abeda\AppData\Local\Programs\Python\Python313\Lib\site-package

In [10]:
import pandas as pd

df = pd.read_csv('AllDataAfterConvertingToVectorsAfterTranslate.csv')

In [11]:
def comprehensive_data_preparation(df, text_column='Name', label_column='Gender'):
    """
    إعداد شامل للبيانات مع معالجة جميع الأخطاء
    """
    print("🚀 بدء الإعداد الشامل للبيانات...")
    
    # الخطوة 1: تنظيف البيانات
    df_clean = df.dropna(subset=[text_column, label_column]).copy()
    print(f"📊 البيانات بعد التنظيف: {len(df_clean)} عينة")
    
    # الخطوة 2: معالجة النصوص
    features_list = []
    labels_list = []
    
    for idx, row in df_clean.iterrows():
        try:
            text = row[text_column]
            label = row[label_column]
            
            # معالجة النص
            if isinstance(text, str):
                if text.startswith('[') and text.endswith(']'):
                    # إذا كان النص عبارة عن قائمة متجهات
                    try:
                        vectors = literal_eval(text)
                        if vectors:
                            arr = np.array(vectors, dtype=np.float32)
                            flattened = arr.flatten()
                            # أخذ أول 500 عنصر أو أقل
                            feature_vector = flattened[:500] if len(flattened) > 500 else flattened
                            # إذا كان أقصر، أضف أصفاراً
                            if len(feature_vector) < 500:
                                feature_vector = np.pad(feature_vector, (0, 500 - len(feature_vector)), 
                                                      mode='constant')
                        else:
                            feature_vector = np.zeros(500, dtype=np.float32)
                    except:
                        feature_vector = np.zeros(500, dtype=np.float32)
                else:
                    # إذا كان نصاً عادياً، استخدم ميزات بسيطة
                    feature_vector = np.array([
                        len(text),
                        sum(1 for c in text if c.isalpha()),
                        sum(1 for c in text if c.isspace()),
                    ], dtype=np.float32)
                    # جعل الطول 500 بإضافة أصفار
                    if len(feature_vector) < 500:
                        feature_vector = np.pad(feature_vector, (0, 500 - len(feature_vector)), 
                                              mode='constant')
            else:
                feature_vector = np.zeros(500, dtype=np.float32)
            
            features_list.append(feature_vector)
            labels_list.append(label)
            
        except Exception as e:
            print(f"⚠️ تخطي عينة {idx}: {e}")
            continue
    
    # الخطوة 3: تحويل إلى numpy arrays
    X = np.array(features_list)
    y = np.array(labels_list)
    
    print(f"✅ الإعداد الكامل: X.shape={X.shape}, y.shape={y.shape}")
    
    return X, y

# الاستخدام
df = df.sample(frac=1).reset_index(drop=True)
X_final, y_final = comprehensive_data_preparation(df)

🚀 بدء الإعداد الشامل للبيانات...
📊 البيانات بعد التنظيف: 25083 عينة
✅ الإعداد الكامل: X.shape=(25083, 500), y.shape=(25083,)


In [12]:
def safe_embedding_processing(series, strategy='flatten'):
    """
    معالجة آمنة للبيانات غير المتجانسة
    """
    # الخطوة 1: جمع جميع الأشكال
    all_shapes = []
    valid_data = []
    
    for item in series:
        if isinstance(item, str):
            try:
                from ast import literal_eval
                item = literal_eval(item)
            except:
                continue
        
        arr = np.array(item, dtype=np.float32)
        all_shapes.append(arr.shape)
        valid_data.append(arr)
    
    print(f"الأشكال الموجودة: {set(all_shapes)}")
    
    # الخطوة 2: إيجاد أقصى طول
    max_length = 0
    for arr in valid_data:
        length = arr.size  # العدد الكلي للعناصر
        if length > max_length:
            max_length = length
    
    print(f"أقصى طول: {max_length}")
    
    # الخطوة 3: جعل جميع المتجهات بنفس الطول
    processed_data = []
    for arr in valid_data:
        flattened = arr.flatten()
        current_length = flattened.shape[0]
        
        if current_length < max_length:
            # إضافة أصفار
            padded = np.pad(flattened, (0, max_length - current_length), 
                          mode='constant', constant_values=0)
            processed_data.append(padded)
        else:
            # إذا كان الطول مناسباً
            processed_data.append(flattened[:max_length])
    
    # الخطوة 4: تحويل إلى numpy array
    try:
        result_array = np.array(processed_data)
        print(f"الشكل النهائي: {result_array.shape}")
        return result_array
    except Exception as e:
        print(f"خطأ في التحويل: {e}")
        # إذا فشل، إرجاع كقائمة
        return processed_data

In [13]:
xof = safe_embedding_processing(df['Name'])

الأشكال الموجودة: set()
أقصى طول: 0
الشكل النهائي: (0,)


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final)

In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import joblib

model = SGDClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))

joblib.dump(model, 'Partial_Model.joblib')

0.5364375697655877


['Partial_Model.joblib']

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# تحميل البيانات
df = pd.read_csv("AllDataAfterTranslate.csv")

# تنظيف بسيط
df['Name'] = df['Name'].str.strip()

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(
    df['Name'], df['Gender'], test_size=0.2, random_state=42
)

# تمثيل النص
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [6]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier()
model.fit(X_train_vec, y_train)

# تقييم
y_pred = model.predict(X_test_vec)
print(f"الدقة: {accuracy_score(y_test, y_pred)}")

الدقة: 0.7923061590591988


In [8]:
import joblib

joblib.dump(model, 'name_gender_model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')

['vectorizer.joblib']

In [24]:
model.predict(vectorizer.transform(['حمدي']))

array([1])

In [14]:
import numpy as np

def preprocess_for_model_fixed(name, target_features=500):
    """
    نسخة مصححة تماماً - تضمن إرجاع شكل 2D
    """
    print(f"🔍 معالجة الاسم: '{name}'")
    
    # المعالجة الأصلية
    vectors = preprocess(name)
    print(f"   عدد متجهات الكلمات: {len(vectors) if vectors else 0}")
    
    if not vectors:
        # إذا لم يكن هناك متجهات، إنشاء متجه صفري بالحجم المطلوب
        processed_vector = np.zeros(target_features, dtype=np.float32)
        print(f"   ⚠️  لا توجد متجهات، استخدام متجه صفري")
    else:
        # تحويل إلى numpy array
        vectors_array = np.array(vectors, dtype=np.float32)
        print(f"   شكل متجهات الكلمات: {vectors_array.shape}")
        
        # تسطيح المتجهات
        flattened = vectors_array.flatten()
        print(f"   شكل المتجه المسطح: {flattened.shape}")
        
        # جعل الطول ثابتاً (500 ميزة)
        if len(flattened) < target_features:
            # إضافة أصفار إذا كان أقصر
            processed_vector = np.pad(flattened, (0, target_features - len(flattened)), 
                                   mode='constant', constant_values=0)
            print(f"   ➕ تمت إضافة أصفار: {len(flattened)} → {target_features}")
        elif len(flattened) > target_features:
            # تقليم إذا كان أطول
            processed_vector = flattened[:target_features]
            print(f"   ✂️  تم التقليم: {len(flattened)} → {target_features}")
        else:
            processed_vector = flattened
            print(f"   ✅ الطول مناسب: {target_features}")
    
    # ⭐⭐ التحويل إلى 2D - هذا هو الحل الرئيسي ⭐⭐
    processed_vector_2d = processed_vector.reshape(1, -1)
    print(f"   📊 الشكل النهائي (2D): {processed_vector_2d.shape}")
    
    return processed_vector_2d

In [23]:
def predict_gender_fixed(name, model, scaler=None, target_features=500):
    """
    نسخة مصححة تماماً من دالة التنبؤ
    """
    print(f"\n🎯 بدء التنبؤ للاسم: '{name}'")
    
    try:
        # استخدام الدالة المصححة التي تُرجع 2D
        X_new = preprocess_for_model_fixed(name, target_features)
        
        # التأكد من أن الشكل صحيح
        if X_new.ndim != 2:
            print(f"❌ خطأ: البيانات ليست 2D، الشكل: {X_new.shape}")
            X_new = X_new.reshape(1, -1)
            print(f"   تم التصحيح إلى: {X_new.shape}")
        
        print(f"📦 شكل البيانات المدخلة للنموذج: {X_new.shape}")
        
        # تطبيق التطبيع إذا كان متوفراً
        if scaler is not None:
            print("🔧 تطبيق التطبيع...")
            X_new = scaler.transform(X_new)
        
        # التنبؤ
        print("🤖 جاري التنبؤ...")
        prediction = model.predict(X_new)[0]
        print(f"   التنبؤ الخام: {prediction}")
        
        # الحصول على الاحتمالات إذا كانت متاحة
        probability = None
        if hasattr(model, 'predict_proba'):
            probability = model.predict_proba(X_new)[0]
            print(f"   الاحتمالات: {probability}")
        
        # تفسير النتيجة
        gender = "ذكر" if prediction == 1 else "أنثى"
        
        result = {
            'name': name,
            'gender': gender,
            'prediction': prediction
        }
        
        if probability is not None:
            result['probability'] = {
                'female_prob': float(probability[0]),
                'male_prob': float(probability[1])
            }
            result['confidence'] = float(max(probability))
        
        print(f"✅ النتيجة النهائية: {gender}")
        return result
        
    except Exception as e:
        print(f"❌ خطأ في التنبؤ: {e}")
        return {
            'name': name,
            'error': str(e),
            'gender': 'غير معروف'
        }

In [28]:
predict_gender_fixed('وردة', model)


🎯 بدء التنبؤ للاسم: 'وردة'
🔍 معالجة الاسم: 'وردة'
   عدد متجهات الكلمات: 1
   شكل متجهات الكلمات: (1, 100)
   شكل المتجه المسطح: (100,)
   ➕ تمت إضافة أصفار: 100 → 500
   📊 الشكل النهائي (2D): (1, 500)
📦 شكل البيانات المدخلة للنموذج: (1, 500)
🤖 جاري التنبؤ...
   التنبؤ الخام: 1
✅ النتيجة النهائية: ذكر


{'name': 'وردة', 'gender': 'ذكر', 'prediction': np.int64(1)}